In [ ]:
import pygame
import numpy as np
from Game import GameEnvironment
from model import QNetwork, get_network_input
import torch
import sys

import cv2

gridsize = 23 
framerate = 10
block_size = 30 

snake_name = 'Snake_59000'

model = QNetwork(input_dim=10, hidden_dim=20, output_dim=5)
model.load_state_dict(torch.load('./dir_chk_lr0.00001/' + snake_name))

board = GameEnvironment(gridsize, nothing=0, dead=-1, apple=1)
windowwidth = gridsize*block_size*2
windowheight = gridsize*block_size

pygame.init()
win = pygame.display.set_mode((windowwidth, windowheight))
pygame.display.set_caption("snake")
font = pygame.font.SysFont('Helvetica', 14)
clock = pygame.time.Clock()


VIDEO = [] # string buffer

def drawboard(snake, apple):
    win.fill((0,0,0))
    for pos in snake.prevpos:
        pygame.draw.rect(win, (0,255,0), (pos[0]*block_size, pos[1]*block_size, block_size, block_size))
    pygame.draw.rect(win, (255, 0, 0), (apple.pos[0]*block_size, apple.pos[1]*block_size, block_size, block_size))

runGame = True

prev_len_of_snake = 0

while runGame:
    clock.tick(framerate)
    
    state_0 = get_network_input(board.snake, board.apple)
    state = model(state_0)
    
    action = torch.argmax(state)
    
    reward, done, len_of_snake = board.update_boardstate(action)
    drawboard(board.snake, board.apple)
    
    lensnaketext     = font.render('          LEN OF SNAKE: ' + str(len_of_snake), False, (255, 255, 255))
    prevlensnaketext = font.render('          LEN OF PREVIOUS SNAKE: ' + str(prev_len_of_snake), False, (255, 255, 255))
    
    x_pos= (int)(0.75*windowwidth)
    win.blit(lensnaketext, (x_pos, 40))
    win.blit(prevlensnaketext, (x_pos, 80))
        
    VIDEO.append(pygame.image.tostring(win, 'RGB', False))     
    
    for event in pygame.event.get():
        if event.type==pygame.KEYDOWN and event.key == pygame.K_ESCAPE:   
            runGame = False
            
    keys = pygame.key.get_pressed()
    if keys[pygame.K_r]:
        paused = True
        while paused == True:
            clock.tick(10)
            pygame.event.pump()
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN:
                    paused = False
    
    pygame.display.update()
    
    if board.game_over == True:
        prev_len_of_snake = len_of_snake
        board.resetgame()

fourcc = cv2.VideoWriter_fourcc(*'MPV4')
output_name = 'output_' + snake_name + '.mp4'
video_mp4 = cv2.VideoWriter(output_name,fourcc, 20.0, (windowwidth,windowheight))        
    
for image in VIDEO:
    
    image = np.frombuffer(image, np.uint8).reshape(windowheight, windowwidth, 3)
    image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)
    video_mp4.write(image)

cv2.destroyAllWindows()
video_mp4.release()
    
pygame.quit()            